<a href="https://colab.research.google.com/github/Desi-Noviana/TUGASAKHIRFGA/blob/main/TUGAS_AKHIR_COVID19_ACEH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Memanggil API dan Library yang dibutuhkan

In [ ]:
# Mengakses API
import requests
resp_aceh = requests.get('https://data.covid19.go.id/public/api/prov_detail_ACEH.json')
#Mengekstrak Respon
cov_aceh_raw = resp_aceh.json()
print(resp_aceh)

# Analisis Data

In [ ]:
#Analisis Data
print('Nama-nama elemen utama:\n', cov_aceh_raw.keys())
print('\nJumlah total kasus COVID-19 di ACEH : %d' %cov_aceh_raw['kasus_total'])
print('Persentase kematian akibat COVID-19 di ACEH : %f%%' %cov_aceh_raw['meninggal_persen'])
print('Persentase tingkat kesembuhan dari COVID-19 di ACEH : %f%%' %cov_aceh_raw['sembuh_persen'])

In [ ]:
cov_aceh_raw #Menampilkan seluruh data covid19 ACEH

# Menapilkan Informasi Data Yang Lebih Lengkap

In [ ]:
#mendapatkan informasi yang lebih lengkap
import numpy as np
import pandas as pd
cov_aceh = pd.DataFrame(cov_aceh_raw['list_perkembangan'])
print('Info cov_aceh:\n', cov_aceh.info())
print('\nLima data teratas cov_aceh:\n', cov_aceh.head())

In [ ]:
#menjinakkan data/Manipulasi data
cov_aceh_tidy = (cov_aceh.drop(columns=[item for item in cov_aceh.columns
                                          if item.startswith('AKUMULASI')
                                          or item.startswith('DIRAWAT')])
.rename(columns=str.lower)
.rename(columns={'kasus': 'kasus_baru'})
)
cov_aceh_tidy['tanggal'] = pd.to_datetime(cov_aceh_tidy['tanggal']*1e6, unit='ns')
print('Lima data teratas:\n', cov_aceh_tidy.head())

In [ ]:
cov_aceh_tidy.shape
# %%
cov_aceh_tidy.isnull().sum()
# Disimpan dalam bentuk format csv
cov_aceh_tidy.to_csv('D:\\downloads\\DATASET-DATASCIENCE-20211015T124508Z-001\\Analisis Data Covid di Indonesia\\data-API-Aceh.csv')

In [ ]:
#visualisasi data
import matplotlib.pyplot as plt

plt.clf()
fig, ax = plt.subplots(figsize=(10,5))
ax.bar(data=cov_aceh_tidy, x='tanggal', height='kasus_baru')
plt.show()

In [ ]:
#grafik kasus baru
import matplotlib.dates as mdates
fig, ax = plt.subplots(figsize=(10,5))
ax.bar(data=cov_aceh_tidy, x='tanggal', height='kasus_baru', color='orange')
fig.suptitle('Kasus Harian Positif COVID-19 di ACEH', 
             y=1.00, fontsize=16, fontweight='bold', ha='center',  color='green')
ax.set_title('\n Terjadi pelonjakan kasus di Pertengahan bulan Agustus-September 2021',
             fontsize=10, ha='center')
ax.set_xlabel('')
ax.set_ylabel('Jumlah kasus')
ax.text(1, -0.3, 'Sumber data: covid.19.go.id', color='blue',
        ha='right', transform=ax.transAxes)
ax.set_xticklabels(ax.get_xticks(), rotation=90)

ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))

plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [ ]:
#grafik kasus sembuh
fig, ax = plt.subplots(figsize=(10,5))
ax.bar(data=cov_aceh_tidy, x='tanggal', height='sembuh', color='green')
ax.set_title('Kasus Harian Sembuh Dari COVID-19 di ACEH',
fontsize=22, color='purple')
ax.set_xlabel('')
ax.set_ylabel('Jumlah kasus')
ax.text(1, -0.3, 'Sumber data: covid.19.go.id', color='blue',
        ha='right', transform=ax.transAxes)
ax.set_xticklabels(ax.get_xticks(), rotation=90)

ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))

plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [ ]:
#grafik kasus meninggal
plt.clf()
fig, ax = plt.subplots(figsize=(10,5))
ax.bar(data=cov_aceh_tidy, x='tanggal', height='meninggal', color='red')
ax.set_title('Kasus Harian Meninggal Dari COVID-19 di ACEH',
fontsize=22)
ax.set_xlabel('')
ax.set_ylabel('Jumlah kasus')
ax.text(1, -0.3, 'Sumber data: covid.19.go.id', color='blue',
        ha='right', transform=ax.transAxes)
ax.set_xticklabels(ax.get_xticks(), rotation=90)

ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))

plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [ ]:
#cek pekan ini
cov_aceh_pekanan = (cov_aceh_tidy.set_index('tanggal')['kasus_baru']
					  .resample('W')
					  .sum()
					  .reset_index()
					  .rename(columns={'kasus_baru': 'jumlah'})
                    )
cov_aceh_pekanan['tahun'] = cov_aceh_pekanan['tanggal'].apply(lambda x: x.year)
cov_aceh_pekanan['pekan_ke'] = cov_aceh_pekanan['tanggal'].apply(lambda x: x.weekofyear)
cov_aceh_pekanan = cov_aceh_pekanan[['tahun', 'pekan_ke', 'jumlah']].sort_values(['tahun', 'pekan_ke'], ascending=[False, False])

print('Info cov_jatim_pekanan:')
print(cov_aceh_pekanan.info())
print('\nLima data teratas cov_jatim_pekanan:\n', cov_aceh_pekanan)

In [ ]:
cov_aceh_pekanan['jumlah_pekanlalu'] = cov_aceh_pekanan['jumlah'].shift().replace(np.nan, 0).astype(np.int)
cov_aceh_pekanan['lebih_baik'] = cov_aceh_pekanan['jumlah'] < cov_aceh_pekanan['jumlah_pekanlalu']

print('Sepuluh data teratas:\n', cov_aceh_pekanan.head(10))

# download data dalam format csv
cov_aceh_pekanan.to_csv('D:\\downloads\\DATASET-DATASCIENCE-20211015T124508Z-001\\Analisis Data Covid di Indonesia\\data-API-ACEH-pekanan.csv')

In [ ]:
#bar chart
plt.clf()
jml_tahun_terjadi_covid19 = cov_aceh_pekanan['tahun'].nunique()
tahun_terjadi_covid19 = cov_aceh_pekanan['tahun'].unique()
fig, axes = plt.subplots(nrows=jml_tahun_terjadi_covid19,
						 figsize = (10,3*jml_tahun_terjadi_covid19))

fig.suptitle('Kasus Pekanan Positif COVID-19 di ACEH',
			 y=1.00, fontsize=16, fontweight='bold', ha='center')
for i, ax in enumerate(axes):
	ax.bar(data=cov_aceh_pekanan.loc[cov_aceh_pekanan['tahun']==tahun_terjadi_covid19[i]],
	   x='pekan_ke', height='jumlah',
	   color=['green' if x is True else 'yellow'
		 	for x in cov_aceh_pekanan['lebih_baik']])
	if i == 0:
		 ax.set_title('\nKolom hijau menunjukan penambahan kasus baru lebih sedikit dibandingkan satu pekan sebelumnya',
					   fontsize=10)
	elif i == jml_tahun_terjadi_covid19-1:
			ax.text(1, -0.2, 'Sumber data: covid.19.go.id', color='blue',
					ha='right', transform=ax.transAxes)
					   
	ax.set_xlim([0, 52.5])
	ax.set_ylim([0, max(cov_aceh_pekanan['jumlah'])])
	ax.set_xlabel(' ')
	ax.set_ylabel('Jumlah kasus %d'%(tahun_terjadi_covid19[i],))
	ax.grid(axis='y')

plt.tight_layout()
plt.show()

In [ ]:
#pola dan dinamika
cov_aceh_akumulasi = cov_aceh_tidy[['tanggal']].copy()
cov_aceh_akumulasi['akumulasi_aktif'] = (cov_aceh_tidy['kasus_baru'] - cov_aceh_tidy['sembuh'] - cov_aceh_tidy['meninggal']).cumsum()
cov_aceh_akumulasi['akumulasi_sembuh'] = cov_aceh_tidy['sembuh'].cumsum()
cov_aceh_akumulasi['akumulasi_meninggal'] = cov_aceh_tidy['meninggal'].cumsum()
print(cov_aceh_akumulasi.tail())


# download data pola dinamika dalam format csv
cov_aceh_akumulasi.to_csv('D:\\downloads\\DATASET-DATASCIENCE-20211015T124508Z-001\\Analisis Data Covid di Indonesia\\data-API-ACEH-akumulasi.csv')

In [ ]:
#Line Chart
plt.clf()
fig, ax = plt.subplots(figsize=(10,5))
ax.plot('tanggal', 'akumulasi_aktif', data=cov_aceh_akumulasi, lw=2)

ax.set_title('Akumulasi aktif COVID-19 di ACEH',
             fontsize=22)
ax.set_xlabel('')
ax.set_ylabel('Akumulasi kejadian')
ax.text(1, -0.3, 'Sumber data: covid.19.go.id', color='blue',
        ha='right', transform=ax.transAxes)
ax.set_xticklabels(ax.get_xticks(), rotation=90)

ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))

plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
plt.clf()
fig, ax = plt.subplots(figsize=(10,5))
cov_aceh_akumulasi_ts = cov_aceh_akumulasi.set_index('tanggal')
cov_aceh_akumulasi_ts.plot(kind='line', ax=ax, lw=3, color=['blue','green','red'])

ax.set_title('Akumulasi Kasus COVID-19 di ACEH', fontsize=22)
ax.set_xlabel('')
ax.set_ylabel('Akumulasi kejadian')
ax.text(1, -0.3, 'Sumber data: covid.19.go.id', color='blue', ha='right', transform=ax.transAxes)

plt.grid()
plt.tight_layout()
plt.show()			


# %%
cov_aceh_akumulasi_ts.to_csv('D:\\downloads\\DATASET-DATASCIENCE-20211015T124508Z-001\\Analisis Data Covid di Indonesia\\data-API-ACEH-Akumulasi-ts.csv')

#KESIMPULAN


*   Kasus positif harian mengalami pelonjakan pada pertengahan bulan Juli hingga Agustus-September 2021
*   Kasus Sembuh Harian mengalami peningkatan pada pertengahan bulan Agustus 2021 dan pertengahan September 2021
* Kasus Meninggal Harian Mengalami peningkatan dimulai dari bulan Agustus hingga September 2021
* Dalam dinamika kasus covid19 ACEH angka peningkatan kesembuhan pasien lebih banyak meningkat dari kasus aktif dan meninggal



